In [31]:
# Python ≥3.5 is required
import sys

import keras.datasets.mnist

print("Python: ", sys.version_info)
assert sys.version_info >= (3, 7)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.8 is required
import tensorflow as tf
print("TF version: ", tf.__version__)
# assert tf.__version__ >= "2.8"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Common imports
import numpy as np
import os
import pandas as pd
import tensorflow_datasets as tfds
from tensorflow_datasets.core.utils import gcs_utils
gcs_utils._is_gcs_disabled = True


# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

Python:  sys.version_info(major=3, minor=9, micro=19, releaselevel='final', serial=0)
sklearn version:  1.4.1.post1
TF version:  2.15.0
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs


In [2]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "deep"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    savefig = plt.savefig(path, format=fig_extension, dpi=resolution)

In [3]:
! pip install --upgrade tfds-nightly

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


## get the data(EMNIST)

In [4]:
import tensorflow_datasets as tfds

(train_data, valid_data, test_data), info = tfds.load("emnist/byclass", split=["train[:80%]", "train[80%:]", "test"], as_supervised=True, with_info=True)

In [8]:
print(f"size of train: {train_data.cardinality().numpy()}")
print(f"size of validation: {valid_data.cardinality().numpy()}")
print(f"size of test: {test_data.cardinality().numpy()}")

size of train: 558346
size of validation: 139586
size of test: 116323


In [15]:
num_class = info.features["label"].num_classes
print(num_class)

62


## prepare the data

In [9]:
def preprocess_lenet5(image, label):
    # Resize image to (32, 32, 1)
    image = tf.image.resize(image, size=(32, 32))
    # Rescale pixel values to [0, 1]
    image = tf.cast(image, tf.float64) / 255.0
    return image, label

In [10]:
train_data = train_data.map(preprocess_lenet5)
valid_data = valid_data.map(preprocess_lenet5)
test_data = test_data.map(preprocess_lenet5)

In [11]:
# Batch and prefetch the datasets
batch_size = 32
train_data = train_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
valid_data = valid_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

## train / evaluate lenet5

In [27]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, AvgPool2D

class LeNet(Sequential):
    def __init__(self,num_classes):
        super().__init__()

        self.add(Conv2D(6,5,strides=1,activation='relu',input_shape=(32,32,1),padding='same'))
        self.add(AvgPool2D(2,strides=2))
        self.add(Conv2D(16,5,strides=1,activation='relu'))
        self.add(AvgPool2D(2,strides=2))
        self.add(Conv2D(120,5,strides=1,activation='relu'))
        self.add(Flatten())
        self.add(Dense(84,activation='tanh'))
        self.add(Dense(num_classes,activation='softmax'))
    
        self.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics='accuracy')

In [28]:
model = LeNet(num_class)
model.summary()

Model: "le_net_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 6)         156       
                                                                 
 average_pooling2d_8 (Avera  (None, 16, 16, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_13 (Conv2D)          (None, 12, 12, 16)        2416      
                                                                 
 average_pooling2d_9 (Avera  (None, 6, 6, 16)          0         
 gePooling2D)                                                    
                                                                 
 conv2d_14 (Conv2D)          (None, 2, 2, 120)         48120     
                                                                 
 flatten_4 (Flatten)         (None, 480)               0  

In [29]:
history = model.fit(train_data, epochs=3, validation_data=valid_data, batch_size=128)

Epoch 1/3
17449/17449 [==============================] - 95s 5ms/step - loss: 0.5302 - accuracy: 0.8201 - val_loss: 0.4206 - val_accuracy: 0.8475
Epoch 2/3
17449/17449 [==============================] - 92s 5ms/step - loss: 0.4038 - accuracy: 0.8524 - val_loss: 0.4041 - val_accuracy: 0.8516
Epoch 3/3
17449/17449 [==============================] - 91s 5ms/step - loss: 0.3806 - accuracy: 0.8589 - val_loss: 0.4023 - val_accuracy: 0.8540


## evaluate the model

In [30]:
model.evaluate(test_data)

3636/3636 [==============================] - 11s 3ms/step - loss: 0.4044 - accuracy: 0.8529


[0.4044412672519684, 0.8529353737831116]

TODO:
1. callback 추가(earlystopping 등)
2. tensorboard로 model training 시각화(교재 참고) 